In [2]:
from http import HTTPStatus
import dashscope
import numpy as np

api = 'sk-c38104258eae4578b8cdbd27c32d5ba9'  # 替换为API KEY
dashscope.api_key = api

# Load data
token_path = './data/csi_token.npy'
label_path = './data/people.npy'

tokens = np.load(token_path)
labels = np.load(label_path)

# Set prompt for training
prompt_train = '''You are a classifier. I have a series of tokens which belong to four labels.
You will be given multiple tokens and corresponding labels in our following dialogues.
In every round of dialogue, I will give you 13 pairs of tokens and labels and you need to learn them.
If you have learned these two kinds of data well,you only need to answer directly with numbers how many pairs of 
tokens and labels you have learned from all the previous dialogues so far,no any other words!!!only a single number'''

# Batch size for training in each round
batch_size = 12
# Create messages for the batch
messages = [{'role': 'system', 'content': prompt_train}]

# Loop through your data for training with batch_size
for batch_start in range(0, 24, batch_size):
    batch_end = min(batch_start + batch_size, 24)
    
    # Prepare batch data
    batch_tokens = tokens[batch_start:batch_end]
    batch_labels = labels[batch_start:batch_end]

    # Construct a single user message with information from all data points
    user_content = '\n'.join([f'This is the token{i+batch_start}: {str(batch_tokens[i,:,:])}. And this is its label{i+batch_start}: {str(batch_labels[i])},I have given you {i+batch_start+1} pairs of tokens and labels.' for i in range(len(batch_tokens))])
    #print(user_content)
    user_message = {'role': 'user', 'content': user_content}
    messages.append(user_message)

    # Call the DashScope API for training
    response = dashscope.Generation.call(
        model='llama2-13b-chat-v2',
        messages=messages,
        result_format='message',
    )

    if response.status_code != HTTPStatus.OK:
        print(f'Request id: {response.request_id}, Status code: {response.status_code}, Error code: {response.code}, Error message: {response.message}')
    else:
        # Check if 'choices' is present in the response
        if 'choices' in response.output:
            # Optionally, you can append the model's response to the messages list
            response_message = {'role': response.output.choices[0]['message']['role'],
                                'content': response.output.choices[0]['message']['content']}
            messages.append(response_message)
            print(response)
        else:
            print(f'Error: Model response does not contain "choices" attribute.')

# Training completed
print('Llama model training completed.')


{"status_code": 200, "request_id": "6b91d18f-9e36-9694-97f0-ab85d0f80a2a", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": " Sure! I have learned 12 pairs of tokens and labels so far.", "content_type": "text"}}]}, "usage": {"input_tokens": 1972, "output_tokens": 18}}
{"status_code": 200, "request_id": "f56e9bd8-aa1d-9a1c-b29a-3e38749614cd", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": " Sure! I have learned 24 pairs of tokens and labels so far.", "content_type": "text"}}]}, "usage": {"input_tokens": 3818, "output_tokens": 18}}
Llama model training completed.
